In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as st
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/dataset/ecommerce/Online Retail.xlsx')

In [ ]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
66579,541809,20677,PINK POLKADOT BOWL,2,2011-01-21 14:59:00,2.46,NaN,United Kingdom
335792,566291,22624,IVORY KITCHEN SCALES,1,2011-09-11 15:31:00,8.50,12965.0,United Kingdom
22789,538177,22190,LOCAL CAFE MUG,1,2010-12-10 09:51:00,2.51,NaN,United Kingdom
14389,537534,20726,LUNCH BAG WOODLAND,1,2010-12-07 11:48:00,0.00,NaN,United Kingdom
144222,548732,21509,COWBOYS AND INDIANS BIRTHDAY CARD,12,2011-04-04 10:16:00,0.42,16142.0,United Kingdom


## Data Understanding

In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Top 20 Product

In [ ]:
top20 = df.groupby('Description')['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False).head(20)
fig = px.bar(top20, x='Description', y='Quantity')
fig.show()

#### Top 20 Canceled Product

In [ ]:
top20_c = df.groupby('Description', as_index=False)['Quantity']\
            .apply(lambda x: x[x<0].sum())\
            .sort_values('Quantity', ascending=True).head(20)
fig = px.bar(top20_c, x='Description', y='Quantity')
fig.show()

### Country

In [ ]:
country = df['Country'].value_counts().reset_index().sort_values('Country', ascending=False)
fig = px.bar(country, x='index', y='Country')
fig.update_xaxes(tickangle=-90)
fig.show()

#### Without UK

In [ ]:
country = df['Country'].value_counts().reset_index()[1:].sort_values('Country', ascending=False)
fig = px.bar(country, x='index', y='Country')
fig.update_xaxes(tickangle=-90)
fig.show()

### Invoice Date

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df['Day'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour
df['Month'] = df['InvoiceDate'].dt.month

#### Day

In [ ]:
day = df['Day'].value_counts().reset_index().sort_values('index', ascending=False)
fig = px.bar(day, x='index', y='Day')
fig.show()

#### Hour

In [ ]:
hour = df['Hour'].value_counts().reset_index().sort_values('index', ascending=False)
fig = px.bar(hour, x='index', y='Hour')
fig.show()

In [ ]:
day_hour = df.groupby(['Day','Hour'])['InvoiceNo'].count().unstack().fillna(0)
fig = px.imshow(day_hour, x=day_hour.columns, y=day_hour.index)
fig.show()

## Data Preparation

#### Remove missing value

In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
Day                 0
Hour                0
Month               0
dtype: int64

In [ ]:
df = df.dropna(subset='CustomerID')

#### Add total price

In [ ]:
df['TotalPrice'] = df['UnitPrice']*df['Quantity']

<ipython-input-17-09f5c04b7015>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## RFM Features

In [ ]:
#Recency
max_date = df['InvoiceDate'].dt.date.max()
r = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
r['InvoiceDate'] = r['InvoiceDate'].dt.date
r['recency'] = r['InvoiceDate'].apply(lambda x: (max_date-x).days)
r = r.drop('InvoiceDate', axis=1)

In [ ]:
#Frequency
f = df.groupby('CustomerID', as_index=False)['InvoiceDate'].nunique()
f.columns = ['CustomerID','frequency']

In [ ]:
#Monetary
m = df.groupby('CustomerID', as_index=False)['TotalPrice'].sum()
m.columns = ['CustomerID','monetary']

In [ ]:
rfm = r.merge(f, on='CustomerID', how='left')
rfm = rfm.merge(m, on='CustomerID', how='left').drop('CustomerID', axis=1)

## Modeling and Evaluation

### Scaling

In [ ]:
cols = rfm.columns.tolist()
rfm_scaled = StandardScaler().fit_transform(rfm)
rfm_scaled = pd.DataFrame(rfm_scaled, columns=cols)
rfm_scaled.head()

,recency,frequency,monetary
0,2.316568,-0.328714,-0.231001
1,-0.889050,0.211970,0.293432
2,-0.164560,-0.112440,-0.012316
3,-0.730258,-0.436851,-0.017146
4,2.167700,-0.436851,-0.190312


### Best K

In [ ]:
sil = []
for i in range(3,15):
    kmeans = KMeans(n_clusters=i, n_init='auto').fit(rfm_scaled)
    preds = kmeans.predict(rfm_scaled)
    euclidean = silhouette_score(rfm_scaled, preds, metric='euclidean')
    print('n_clusters:', i,'. Silhouette score:', euclidean)
    sil.append(euclidean)

n_clusters: 3 . Silhouette score: 0.5814191102146425
n_clusters: 4 . Silhouette score: 0.5986266923232999
n_clusters: 5 . Silhouette score: 0.45170586066755986
n_clusters: 6 . Silhouette score: 0.5150697244664523
n_clusters: 7 . Silhouette score: 0.501239297313388
n_clusters: 8 . Silhouette score: 0.4922248928240906
n_clusters: 9 . Silhouette score: 0.4987679249526803
n_clusters: 10 . Silhouette score: 0.42164720387646903
n_clusters: 11 . Silhouette score: 0.40347516574366526
n_clusters: 12 . Silhouette score: 0.41565808352948824
n_clusters: 13 . Silhouette score: 0.41119846669562815
n_clusters: 14 . Silhouette score: 0.4000491197214699


In [ ]:
fig = px.line(x=range(3,15), y=sil, markers=True)
fig.show()

### Clustering

In [ ]:
kmeans = KMeans(n_clusters=4, n_init=10, random_state=100)
kmeans.fit(rfm_scaled)

KMeans(n_clusters=4, n_init=10, random_state=100)

In [ ]:
rfm['cluster'] = kmeans.labels_
rfm['aov'] = rfm['monetary']/rfm['frequency']

In [ ]:
perc = np.arange(0.1,1,0.1)
rfm.describe(percentiles=perc)

,recency,frequency,monetary,cluster,aov
count,4372.000000,4372.000000,4372.000000,4372.000000,4372.000000
mean,91.581199,5.039799,1898.459701,1.227356,317.302387
std,100.772139,9.248607,8219.345141,0.476395,365.792196
min,0.000000,1.000000,-4287.630000,0.000000,-4287.630000
10%,4.000000,1.000000,146.022000,1.000000,100.126583
20%,11.000000,1.000000,234.392000,1.000000,138.038667
30%,21.000000,1.000000,337.370000,1.000000,167.349875
40%,31.000000,2.000000,465.412000,1.000000,200.131429
50%,50.000000,3.000000,648.075000,1.000000,237.475937
60%,71.000000,4.000000,909.134000,1.000000,288.806857


In [ ]:
rfm_summary = rfm.groupby('cluster').agg({'recency':'mean',
                                          'frequency':'mean',
                                          'monetary':['mean','median'],
                                          'aov':'mean'})
rfm_summary

recency  frequency       monetary                      aov
               mean       mean           mean      median         mean
cluster                                                               
0          8.773585  40.547170   18940.097736   12091.895   554.108695
1         41.066520   4.808008    1483.679591     856.680   323.898986
2        247.386949   1.799632     453.209468     296.815   259.501435
3          7.166667  87.833333  182181.981667  160027.395  3127.561923

0 : potential loyal customer <br>
1 : new customer <br>
2 : lost customer <br>
3 : champion <br>